Run fishbone integrator from $\epsilon=200$.

1. How does it compare with MS-HAN $p_{\text{hug}}\in\{0.1,0.3,0.5\}$? 
2. How does it compare with MS-RWM?

In [1]:
import numpy as np
from numpy import zeros, eye, array, diag, exp, ones, tanh, cosh, dstack, log, hstack, vstack, sinh
from numpy.linalg import solve, norm
from numpy.random import choice, default_rng, randint, randn
from scipy.stats import multivariate_normal as MVN
from scipy.special import logsumexp, ndtr
from scipy.stats import uniform as udist
import math
import time
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import IntSlider
from ipywidgets.widgets import interact
from warnings import resetwarnings, catch_warnings, filterwarnings
import seaborn as sns
import pickle

from Manifolds.GKManifoldNew import GKManifold, data_generator, find_point_on_manifold, find_point_on_manifold_from_θ
from markov_snippets import MSAdaptive, SMCAdaptive
from tangential_hug_functions import TangentialHugSampler
from tangential_hug_functions import HugTangentialMultivariate
from RWM import RWM
from utils import ESS_univariate
from utils import send_notification_to_iphone

seed = 1234
m = 20     # number of latent variables
d = m + 4  # dimensionality of x-component

def generate_settings(
    d=d, 
    N=2000,
    B=20,
    δ=0.01,
    δmin=1e-3,
    δmax=100.0,
    εmin=1e-16,
    min_pm=1e-4,
    maxiter=500,
    verbose=False,
    εs_fixed=None,
    adaptiveε=True,
    adaptiveδ=False,
    z0_manual=None,
    pm_target=0.23,
    pd_target=0.4,
    dm_target=0.01,
    pm_switch=0.1,
    prior_seed=seed,
    low_memory=True,
    integrator='rwm',
    εprop_switch=0.01,
    ε0_manual=None,
    quantile_value=0.9,
    initialization='manual',
    switch_strategy='pm',
    kernel='uniform',
    prior_scale=7,
    thinning=10,
    burnin=100,
    use_autograd=False,
    stopping_criterion={'εmin', 'pm'},
    resampling_seed=1234,
    mh_kernel_seed=1234,
    resampling_scheme='multinomial',
    prop_hug=0.5,
    projection_method='qr',
    metropolised=False,
    δadaptation_method='ap',
    prop_esjd=0.95,
    adaptiveB=False,
    Bmin=5,
    Bmax=100,
    proxy_ap_metric='dm',
    adaptiveN=False,
    NBbudget=None,
    Badaptation_method=None,
    adaptive_prophug=False,
    min_prop_hug=0.1,
    max_prop_hug=0.9):
    # here d represents m+4
    # Generate the manifold
    θ0        = array([3.0, 1.0, 2.0, 0.5])       # True parameter value on U(0, 10) scale.
    ystar     = data_generator(θ0, m, seed=seed)  # Observed data
    q         = MVN(zeros(d), eye(d))             # Proposal distribution for THUG
    resetwarnings()
    manifold  = GKManifold(ystar, kernel_type=kernel, use_autograd=use_autograd)
    # grab arguments
    arguments = locals()
    return arguments

δmin   = 1e-3
min_pm = 1e-4
pm_switch = 0.1

In [2]:
with open("experiment93/second_particles10k.pkl", "rb") as file:
    P1_10k = pickle.load(file)

with open("experiment93/second_particles10k_new.pkl", "rb") as file:
    P2_10k = pickle.load(file)
    
with open("experiment93/second_particles10k_new_new.pkl", "rb") as file:
    P3_10k = pickle.load(file)
    
P_25k = vstack((P1_10k, P2_10k, P3_10k[:5000]))

In [3]:
with open("experiment93/B10N5000_epsilons.pkl", "rb") as file:
    B10N5000_MSRWM_EPSILONS = pickle.load(file)
    
with open("experiment93/B10N5000_ess_smc.pkl", "rb") as file:
    B10N5000_MSRWM_ESS_SMC = pickle.load(file)
    
with open("experiment93/B10N5000_ess.pkl", "rb") as file:
    B10N5000_MSRWM_ESS = pickle.load(file)
    
with open("experiment93/B10N5000_pm.pkl", "rb") as file:
        B10N5000_MSRWM_PM = pickle.load(file)

In [4]:
B10N5k_FISHBONE = MSAdaptive(generate_settings(d=d, N=5000, B=10,
        δ=0.01, δmin=δmin, min_pm=(1/5000), verbose=True, εs_fixed=None, adaptiveε=True, adaptiveδ=False,
        z0_manual=P1_10k[:5000], pm_target=0.23, pm_switch=pm_switch, low_memory=True, integrator='fishbone',
        εprop_switch=0.01, ε0_manual=200.0, quantile_value=0.5, initialization='manual',
        switch_strategy='pm', kernel='uniform',use_autograd=False, stopping_criterion={'εmin', 'pm', 'maxiter'},
        resampling_scheme='multinomial', maxiter=1000, δadaptation_method=None, adaptiveB=False,
        Bmin=5, Bmax=100, proxy_ap_metric='pm', prop_esjd=0.95, Badaptation_method=None, prop_hug=0.1,
        adaptive_prophug=False))
B10N5k_FISHBONE_PARTICLES = B10N5k_FISHBONE.sample()

Integrator: FISHBONE.
Stopping criterion:  maxiter, εmin, pm.
Resampling: MULTINOMIAL.
Iteration:  1
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 55.2105774338382034
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9157999999999999
	Particle Diversity: 0.4568000000000000
	Moved Diversity: 0.4183374399999999
	Step-size kept fixed at: 0.0100000000000000
Iteration:  2
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 32.8329969912970228
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9156000000000000
	Particle Diversity: 0.4344000000000000
	Moved Diversity: 0.3977366400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  3
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 24.8047517148264305
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled

	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9012000000000000
	Particle Diversity: 0.4762000000000000
	Moved Diversity: 0.4291514400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  24
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.7065776405249071
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8952000000000000
	Particle Diversity: 0.4844000000000000
	Moved Diversity: 0.4336348800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  25
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.7061798195882503
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8994000000000000
	Particle Diversity: 0.4744000000000000
	Moved Diversity: 0.4266753600000000
	Step-size kept fixed at: 0.0100000000000000

	Trajectories constructed.
	Epsilon: 13.6976698358533309
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8940000000000000
	Particle Diversity: 0.4686000000000000
	Moved Diversity: 0.4189284000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  47
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6972984383439300
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8940000000000000
	Particle Diversity: 0.4736000000000000
	Moved Diversity: 0.4233984000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  48
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6969298472988648
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8872000000000000
	Particle Diversity: 0.4716000000000000
	Moved Diversity: 0.4184

	Trajectories constructed.
	Epsilon: 13.6891018208345514
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8842000000000000
	Particle Diversity: 0.4700000000000000
	Moved Diversity: 0.4155739999999999
	Step-size kept fixed at: 0.0100000000000000
Iteration:  70
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6887772436641590
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8858000000000000
	Particle Diversity: 0.4706000000000000
	Moved Diversity: 0.4168574800000001
	Step-size kept fixed at: 0.0100000000000000
Iteration:  71
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6884391137576618
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8848000000000000
	Particle Diversity: 0.4706000000000000
	Moved Diversity: 0.4163

	Trajectories constructed.
	Epsilon: 13.6812966319173519
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8885999999999999
	Particle Diversity: 0.4752000000000000
	Moved Diversity: 0.4222627200000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  93
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6809475173787103
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8862000000000000
	Particle Diversity: 0.4718000000000000
	Moved Diversity: 0.4181091600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  94
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6805922564159115
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8776000000000000
	Particle Diversity: 0.4632000000000000
	Moved Diversity: 0.4065

...

	Trajectories constructed.
	Epsilon: 13.6467799006836348
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8972000000000000
	Particle Diversity: 0.4828000000000000
	Moved Diversity: 0.4331681600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  185
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6464202885957153
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9000000000000000
	Particle Diversity: 0.4784000000000000
	Moved Diversity: 0.4305600000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  186
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6460648714360318
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8980000000000000
	Particle Diversity: 0.4716000000000000
	Moved Diversity: 0.42

	Trajectories constructed.
	Epsilon: 13.6367754223318922
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9042000000000000
	Particle Diversity: 0.4824000000000000
	Moved Diversity: 0.4361860800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  208
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6363029915739560
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9086000000000000
	Particle Diversity: 0.4886000000000000
	Moved Diversity: 0.4439419600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  209
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6358233240156448
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8980000000000000
	Particle Diversity: 0.4852000000000000
	Moved Diversity: 0.43

	Trajectories constructed.
	Epsilon: 13.6251618361330316
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8996000000000000
	Particle Diversity: 0.4780000000000000
	Moved Diversity: 0.4300088000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  231
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6246292155171886
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9022000000000000
	Particle Diversity: 0.4866000000000000
	Moved Diversity: 0.4390105200000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  232
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6240719303132387
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9032000000000000
	Particle Diversity: 0.4834000000000000
	Moved Diversity: 0.43

	Trajectories constructed.
	Epsilon: 13.6121772851525744
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8905999999999999
	Particle Diversity: 0.4850000000000000
	Moved Diversity: 0.4319410000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  254
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6116875170224585
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9034000000000000
	Particle Diversity: 0.4822000000000000
	Moved Diversity: 0.4356194800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  255
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.6111801723031505
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9074000000000000
	Particle Diversity: 0.4772000000000000
	Moved Diversity: 0.43

	Trajectories constructed.
	Epsilon: 13.5985595122381948
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9126000000000000
	Particle Diversity: 0.4790000000000000
	Moved Diversity: 0.4371354000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  277
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.5979282094314460
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9092000000000000
	Particle Diversity: 0.4792000000000000
	Moved Diversity: 0.4356886400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  278
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 13.5973305564849056
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9026000000000000
	Particle Diversity: 0.4886000000000000
	Moved Diversity: 0.44